In [46]:
from langchain.schema import SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from operator import itemgetter
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory
from langchain_community.llms import GPT4All
from langchain.schema import messages_from_dict, messages_to_dict
from langchain_experimental.chat_models import Llama2Chat
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import json
from langchain.schema import messages_from_dict, messages_to_dict



# model = GPT4All("./chatModels/mistral-7b-instruct-v0.1.Q4_0.gguf")
callbacks = [StreamingStdOutCallbackHandler()]
model = GPT4All(model="../chatModels/mistral-7b-instruct-v0.1.Q4_0.gguf",
                callbacks=callbacks, verbose=True)

rompt_template = ChatPromptTemplate.from_messages(template_messages)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chat = Llama2Chat(llm=model)
memory = ConversationBufferMemory(return_messages=True)



In [47]:
prompt

ChatPromptTemplate(input_variables=['history', 'input'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful chatbot')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [48]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | chat
)

In [56]:
inputs={"input": "Hi I'm bob"}
response=chain.invoke(inputs)
print(response)
memory.save_context(inputs, {"output": response.content})

content=' Hello Bob! How can I assist you today?'


In [87]:
inputs={"input": "Do you know what's my name"}
response=chain.invoke(inputs)
print(response)
memory.save_context(inputs, {"output": response.content})

content=' Yes, your name is Bob. Is there anything else I can help you with?'


In [88]:
inputs={"input": "tell me a joke about my name"}
response=chain.invoke(inputs)
print(response)
memory.save_context(inputs, {"output": response.content})

content=" Sure! Here's one for you:\n\nWhy did the computer go to therapy? Because it had too many Bob files!"


In [93]:
memory.chat_memory.messages[-1].type

'ai'

In [67]:
memory.chat_memory.messages

[HumanMessage(content="Hi I'm bob"),
 AIMessage(content=' Hello Bob! How can I assist you today?'),
 HumanMessage(content="Hi I'm bob"),
 AIMessage(content=' Hello Bob! How can I assist you today?')]

In [83]:
def memory_to_json(memory,save=False):
    mes_dict=messages_to_dict(memory.chat_memory.messages)
    total_dict={'messages':mes_dict}
    mes_json=json.dumps(total_dict)
    if save:
        with open('memory.json', 'w') as fp:
            json.dump(total_dict, fp)
    return mes_json
def json_to_memory(mes_json):
    mes_dict=json.loads(mes_json)
    mes=messages_from_dict(mes_dict['messages'])
    mem=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=mes),return_messages=True)
    return mem

In [78]:
memory_json=memory_to_json(memory)
memory_json

'{"messages": [{"type": "human", "data": {"content": "Hi I\'m bob", "additional_kwargs": {}, "type": "human", "example": false}}, {"type": "ai", "data": {"content": " Hello Bob! How can I assist you today?", "additional_kwargs": {}, "type": "ai", "example": false}}, {"type": "human", "data": {"content": "Hi I\'m bob", "additional_kwargs": {}, "type": "human", "example": false}}, {"type": "ai", "data": {"content": " Hello Bob! How can I assist you today?", "additional_kwargs": {}, "type": "ai", "example": false}}]}'

In [80]:
new_mem=json_to_memory(memory_json)

In [81]:
new_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(new_mem.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | chat
)

In [82]:
chain.invoke({"input": "Do you know what's my name"})

AIMessage(content=' Yes, your name is Bob. Is there anything else I can help you with?')

In [84]:
memory_to_json(new_mem,save=True)

'{"messages": [{"type": "human", "data": {"content": "Hi I\'m bob", "additional_kwargs": {}, "type": "human", "example": false}}, {"type": "ai", "data": {"content": " Hello Bob! How can I assist you today?", "additional_kwargs": {}, "type": "ai", "example": false}}, {"type": "human", "data": {"content": "Hi I\'m bob", "additional_kwargs": {}, "type": "human", "example": false}}, {"type": "ai", "data": {"content": " Hello Bob! How can I assist you today?", "additional_kwargs": {}, "type": "ai", "example": false}}]}'